Some Verra Projects because of their website We extracted and filtered  the data from https://registry.verra.org/app/search/VCS project type allocation to each project some projects were not included in our verrra data download so we manually did a search in the website downloaded the .kml files for the projects and from them extracted the meta data and preprocessed as per the steps shiwn  in this notebook

In [ ]:
from pykml import parser
from os import path
import pandas as pd
import simplekml
import matplotlib.pyplot as plt
import seaborn as sns  
import ast  
import glob
import numpy as np
from bs4 import BeautifulSoup
import re
from pyproj import Proj, transform
from shapely.geometry import Polygon, shape, mapping
from shapely.ops import transform
from functools import partial
import json
import urllib.parse
from fastkml import kml
import geopandas as gpd
from lxml import etree




### Extracting Data for polygons without Tree information

In [ ]:
import os


kml_directory = '../input/Verra_Projects/TIST_no_tree/'
output_directory = '../input/Verra_Projects/TIST_no_tree/'

def parse_coordinates(coordinate_string):
    coordinate_list = coordinate_string.strip().split()
    return [(float(coordinate.split(',')[1]), float(coordinate.split(',')[0])) for coordinate in coordinate_list]

def parse_description(description):
    if description is None:
        return None
    soup = BeautifulSoup(description, 'html.parser')
    table = soup.find('table')
    data = {}
    if table:
        for row in table.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) > 1:
                key = cells[0].text.strip().replace(':', '')
                value = cells[1].text.strip()
                data[key] = value
    return data

# Looping through all KML files in the directory
for kml_file_path in glob.glob(kml_directory + '*.kml'):
    names = []
    descriptions = []
    coordinates = []
    additional_data = []

    with open(kml_file_path, 'r') as kml_file:
        kml_content = parser.parse(kml_file).getroot()

        # Handling of  different namespaces in km files
        if hasattr(kml_content, 'Document'):
            document = kml_content.Document
        elif hasattr(kml_content, '{http://earth.google.com/kml/2.2}Document'):
            document = kml_content.find('{http://earth.google.com/kml/2.2}Document')
        else:
            continue

        for folder in document.Folder:
            for placemark in folder.Placemark:
                name = placemark.name.text if hasattr(placemark, 'name') else None
                description = parse_description(placemark.description.text) if hasattr(placemark, 'description') and placemark.description.text is not None else None
                if hasattr(placemark, 'Point'):
                    coordinate = parse_coordinates(placemark.Point.coordinates.text)
                elif hasattr(placemark, 'LineString'):
                    coordinate = parse_coordinates(placemark.LineString.coordinates.text)
                else:
                    coordinate = None
                names.append(name)
                descriptions.append(description)
                coordinates.append(coordinate)
                additional_data.append(description)

    # Creating  a DataFrame from the extracted data
    df = pd.DataFrame({
        'Name': names,
        'Description': descriptions,
        'Coordinates': coordinates
    })

    # Adding any additional columns dynamically
    if additional_data:
        for data in additional_data:
            if data:
                for key in data.keys():
                    if key not in df.columns:
                        df[key] = None
                for i, data_dict in enumerate(additional_data):
                    if data_dict:
                        for key, value in data_dict.items():
                            df.at[i, key] = value

  
    output_file_path = os.path.join(output_directory, os.path.basename(kml_file_path).replace('.kml', '.csv'))
    df.to_csv(output_file_path, index=False)

Combining all TIST with no tree  datasets

In [ ]:

folder_path = '../input/Verra_Projects/TIST_no_tree/'


files = os.listdir(folder_path)

# Filtering the list to include only CSV files
csv_files = [file for file in files if file.endswith('.csv')]


dataframes = []


for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)
    dataframes.append(df)


concatenated_df = pd.concat(dataframes, ignore_index=True)
concatenated_df.info()

# # Write the concatenated DataFrame to a new CSV file
# output_file_path = os.path.join(folder_path, 'concatenated_output.csv')
# concatenated_df.to_csv(output_file_path, index=False)

Renaming columns where applicable

In [ ]:
concatenated_df.head()

In [ ]:
# Renaming the columns to follow our naming format in the paper columns section
columns_rename_mapping = {
    'Name': 'project_id_reported',
    'Description': 'description_reported',
    'Coordinates': 'geometry_reported'

   
}


concatenated_df.rename(columns=columns_rename_mapping, inplace=True)
concatenated_df.head()

Adding all other defined columns as null values 

In [ ]:
concatenated_df['site_id_reported'] = ['tist_site_{}'.format(i) for i in range(1, len(concatenated_df) + 1)]
concatenated_df = concatenated_df.assign(species_count_reported=None, species_planted_reported=None, survival_rate_reported=None,trees_planted_reported=None,planting_date_reported=None,site_sqkm=None,country=None)
concatenated_df.info()

In [ ]:

concatenated_df.to_csv('../input/TIST/TIST_Meta_without_trees.csv', index=False)

##  Extracting data from  TIST Polygons with trees and area specified

In [ ]:


kml_directory = '../input/Verra_Projects/TIST_trees/'
output_directory = '../input/Verra_Projects/TIST_trees/'

def parse_coordinates(coordinate_string):
    coordinate_list = coordinate_string.strip().split()
    return [(float(coordinate.split(',')[1]), float(coordinate.split(',')[0])) for coordinate in coordinate_list]

def parse_description(description):
    if description is None:
        return None
    soup = BeautifulSoup(description, 'html.parser')
    table = soup.find('table')
    data = {}
    if table:
        for row in table.find_all('tr'):
            cells = row.find_all('td')
            if len(cells) > 1:
                key = cells[0].text.strip().replace(':', '')
                value = cells[1].text.strip()
                data[key] = value
    return data


if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Looping through all KML files in the directory
for kml_file_path in glob.glob(os.path.join(kml_directory, '*.kml')):
    print(f"Processing file: {kml_file_path}")
    names = []
    descriptions = []
    coordinates = []
    additional_data = []

    try:
        with open(kml_file_path, 'r') as kml_file:
            kml_content = parser.parse(kml_file).getroot()

            # Handling different namespaces
            if hasattr(kml_content, 'Document'):
                document = kml_content.Document
            elif hasattr(kml_content, '{http://earth.google.com/kml/2.2}Document'):
                document = kml_content.find('{http://earth.google.com/kml/2.2}Document')
            else:
                print("No Document found in KML")
                continue

            for folder in document.Folder:
                for placemark in folder.Placemark:
                    name = placemark.name.text if hasattr(placemark, 'name') else None
                    description = parse_description(placemark.description.text) if hasattr(placemark, 'description') and placemark.description.text is not None else None
                    if hasattr(placemark, 'Point'):
                        coordinate = parse_coordinates(placemark.Point.coordinates.text)
                    elif hasattr(placemark, 'LineString'):
                        coordinate = parse_coordinates(placemark.LineString.coordinates.text)
                    else:
                        coordinate = None
                    names.append(name)
                    descriptions.append(description)
                    coordinates.append(coordinate)
                    additional_data.append(description)

      
        df = pd.DataFrame({
            'Name': names,
            'Description': descriptions,
            'Coordinates': coordinates
        })

        print(f"DataFrame created with {len(df)} rows")

        # Adding additional columns dynamically
        if additional_data:
            for data in additional_data:
                if data:
                    for key in data.keys():
                        if key not in df.columns:
                            df[key] = None
                    for i, data_dict in enumerate(additional_data):
                        if data_dict:
                            for key, value in data_dict.items():
                                df.at[i, key] = value

      
        output_file_path = os.path.join(output_directory, os.path.basename(kml_file_path).replace('.kml', '.csv'))
        df.to_csv(output_file_path, index=False)
        print(f"CSV file saved: {output_file_path}")
    except Exception as e:
        print(f"Error processing file {kml_file_path}: {e}")

Concatinating all csvs from tist with trees

In [59]:

folder_path = '../input/Verra_Projects/TIST_trees/'


files = os.listdir(folder_path)

# Filtering the list to include only CSV files
csv_files = [file for file in files if file.endswith('.csv')]


dataframes = []


for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)
    dataframes.append(df)


tist_df = pd.concat(dataframes, ignore_index=True)
tist_df.info()

# # Write the concatenated DataFrame to a new CSV file
# output_file_path = os.path.join(folder_path, 'concatenated_output.csv')
# concatenated_df.to_csv(output_file_path, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47237 entries, 0 to 47236
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          47237 non-null  object 
 1   Description   27404 non-null  object 
 2   Coordinates   47230 non-null  object 
 3   Tract Date    26975 non-null  object 
 4   Quantifier 1  25884 non-null  object 
 5   Quantifier 2  25884 non-null  object 
 6   Best Tract    26975 non-null  object 
 7   Grove Area    26148 non-null  object 
 8   Trees         26414 non-null  object 
 9   Unnamed: 8    0 non-null      float64
dtypes: float64(1), object(9)
memory usage: 3.6+ MB


Cleaning the dataset

In [60]:
df = tist_df

# Iterating through the DataFrame
rows_to_drop = []
for i in range(1, len(df)):
    if df.loc[i, 'Name'] == 'Tract 1':
        df.loc[i-1, 'Coordinates'] = df.loc[i, 'Coordinates']
        rows_to_drop.append(i)

# Dropping the tract1  the rows marked for deletion
df.drop(rows_to_drop, inplace=True)


df.reset_index(drop=True, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29940 entries, 0 to 29939
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          29940 non-null  object 
 1   Description   27404 non-null  object 
 2   Coordinates   29933 non-null  object 
 3   Tract Date    26975 non-null  object 
 4   Quantifier 1  25884 non-null  object 
 5   Quantifier 2  25884 non-null  object 
 6   Best Tract    26975 non-null  object 
 7   Grove Area    26148 non-null  object 
 8   Trees         26414 non-null  object 
 9   Unnamed: 8    0 non-null      float64
dtypes: float64(1), object(9)
memory usage: 2.3+ MB


In [61]:
# Remove the substring "(Tract 1)" from the specified column and strip any leading/trailing whitespace
df['Name'] = df['Name'].str.replace(r'\(Tract 1\)', '', regex=True).str.strip()

df.tail()

,Name,Description,Coordinates,Tract Date,Quantifier 1,Quantifier 2,Best Tract,Grove Area,Trees,Unnamed: 8
29935,2009UG81-CanJovanis R 04,NaN,"[(-0.87944, 29.73545), (-0.879446667, 29.73545...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
29936,2009UG81-CanJovanis R 05,NaN,"[(-0.883873333, 29.735696667), (-0.883871667, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
29937,2009UG81-CanJovanis R 06,NaN,"[(-0.881725, 29.750536667), (-0.881696667, 29....",NaN,NaN,NaN,NaN,NaN,NaN,NaN
29938,2014UG43-Batororo,NaN,"[(-1.245026667, 29.926586667), (-1.244831667, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
29939,2014UG43-Ninsiima,NaN,"[(-1.238755, 29.926093333), (-1.238513333, 29....",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
# For those resulting direct from kml extraction 
def resolve_duplicates(group):
   
    group['non_null_count'] = group.notnull().sum(axis=1)
    group = group.sort_values(by='non_null_count', ascending=False)
    
   
    resolved_row = group.iloc[0].copy()
    
    # Ensuring 'Coordinates' column takes the second value if available
    if len(group) > 1 and pd.notnull(group.iloc[1]['Coordinates']):
        resolved_row['Coordinates'] = group.iloc[1]['Coordinates']
    
    return resolved_row.drop('non_null_count')

# Grouping by 'Name' and apply the resolve_duplicates function
df_resolved = df.groupby('Name').apply(resolve_duplicates).reset_index(drop=True)


df_resolved.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20165 entries, 0 to 20164
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          20165 non-null  object 
 1   Description   20165 non-null  object 
 2   Coordinates   20159 non-null  object 
 3   Tract Date    19887 non-null  object 
 4   Quantifier 1  18798 non-null  object 
 5   Quantifier 2  18798 non-null  object 
 6   Best Tract    19887 non-null  object 
 7   Grove Area    19061 non-null  object 
 8   Trees         19326 non-null  object 
 9   Unnamed: 8    0 non-null      float64
dtypes: float64(1), object(9)
memory usage: 1.5+ MB


/var/folders/_l/gj90rjl90fngb4_8ph5x5jhc0000gn/T/ipykernel_3074/528406570.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_resolved = df.groupby('Name').apply(resolve_duplicates).reset_index(drop=True)


In [63]:
df_resolved.head()

,Name,Description,Coordinates,Tract Date,Quantifier 1,Quantifier 2,Best Tract,Grove Area,Trees,Unnamed: 8
0,2002UG2-KARAMUZI P II,"{'Tract Date': '2009-10-19', 'Quantifier 1': '...","[(-0.55757, 30.176911667), (-0.55759, 30.17689...",2009-10-19,-Tushabemukama Apollo,-Agaba Denis,Tract 2,0.36 Ha,633,NaN
1,2002UG2-KOMUNDA Y I,"{'Tract Date': '2009-10-20', 'Quantifier 1': '...","[(-0.55359, 30.182306667), (-0.553586667, 30.1...",2009-10-20,-Agaba Denis,-Tushabemukama Apollo,Tract 1,0.09 Ha,189,NaN
2,2002UG2-KYEITEMBE MU I,"{'Tract Date': '2009-10-19', 'Quantifier 1': '...","[(-0.54439, 30.177173333), (-0.544375, 30.1771...",2009-10-19,-Agaba Denis,-Tushabemukama Apollo,Tract 2,0.46 Ha,"1,090",NaN
3,2002UG2-Karure,"{'Tract Date': '2009-10-20', 'Quantifier 1': '...","[(-0.55598, 30.179956667), (-0.555988333, 30.1...",2009-10-20,-Tushabemukama Apollo,-Agaba Denis,Tract 2,0.60 Ha,214,NaN
4,2002UG2-Perry Karamuzi,"{'Tract Date': '2014-11-17', 'Quantifier 1': '...","[(-0.556313333, 30.17781), (-0.55633, 30.17784...",2014-11-17,Edward Twesigye,Kyorimpa Millian,Tract 1,1.84200 Ha,125.0,NaN


In [64]:

df_resolved['Grove Area'] = pd.to_numeric(df_resolved['Grove Area'], errors='coerce')


df_resolved['Grove Area'].fillna(0, inplace=True)


df_resolved['Grove area (km²)'] = df_resolved['Grove Area'] / 100

/var/folders/_l/gj90rjl90fngb4_8ph5x5jhc0000gn/T/ipykernel_3074/2964322643.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_resolved['Grove Area'].fillna(0, inplace=True)


Rename the columns to align with the general format

In [65]:
# Renaming the columns to follow our naming format in the paper columns section
columns_rename_mapping = {
    'Name': 'project_id_reported',
      'Name': 'site_id_reported',
    'Trees': 'trees_planted_reported',
    'Description': 'description_reported',
    'Tract Date': 'planting_date_reported',
    'Coordinates': 'geometry_reported',
    'Grove area (km²)': 'site_sqkm',
    #'survivalRate': 'survival_rate_reported',
}

df_resolved = df_resolved[list(columns_rename_mapping.keys())]

# Rename the columns
df_resolved.rename(columns=columns_rename_mapping, inplace=True)

#### Expanding the columns by creating empty columns for variables not represented in the meta data

In [66]:
# df_resolved['site_id_reported'] = ['restor_site_{}'.format(i) for i in range(1, len(df_resolved) + 1)]
df_resolved = df_resolved.assign(species_count_reported=None, species_planted_reported=None, survival_rate_reported=None,trees_planted_reported=None,Country=None)
df_resolved.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20165 entries, 0 to 20164
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Site_id_reported          20165 non-null  object 
 1   trees_planted_reported    0 non-null      object 
 2   description_reported      20165 non-null  object 
 3   planting_date_reported    19887 non-null  object 
 4   geometry_reported         20159 non-null  object 
 5   site_sqkm                 20165 non-null  float64
 6   species_count_reported    0 non-null      object 
 7   species_planted_reported  0 non-null      object 
 8   survival_rate_reported    0 non-null      object 
 9   Country                   0 non-null      object 
dtypes: float64(1), object(9)
memory usage: 1.5+ MB


In [ ]:
#Saving The  data to csv
df_resolved.to_csv('../input/TIST/TIST_Meta.csv', index=False)

### Concatinating Data for polygons without Tree information

### Komaza/Coastal Kenya Projects

In [ ]:
from pykml import parser
import pandas as pd
from bs4 import BeautifulSoup
import glob
import os


kml_directory = '../input/Verra_Projects/komaza/'
output_directory = '../input/Verra_Projects/komaza/'

def parse_coordinates(coordinate_string):
    coordinate_list = coordinate_string.strip().split()
    return [(float(coordinate.split(',')[1]), float(coordinate.split(',')[0])) for coordinate in coordinate_list]

def parse_extended_data(extended_data):
    data = {}
    for schema_data in extended_data.SchemaData:
        for simple_data in schema_data.SimpleData:
            data[simple_data.attrib['name']] = simple_data.text
    return data


if not os.path.exists(output_directory):
    os.makedirs(output_directory)


for kml_file_path in glob.glob(os.path.join(kml_directory, '*.kml')):
    print(f"Processing file: {kml_file_path}")
    names = []
    descriptions = []
    coordinates = []
    additional_data = []

    try:
        with open(kml_file_path, 'r') as kml_file:
            kml_content = parser.parse(kml_file).getroot()

            # Handling different namespaces
            if hasattr(kml_content, 'Document'):
                document = kml_content.Document
            elif hasattr(kml_content, '{http://earth.google.com/kml/2.2}Document'):
                document = kml_content.find('{http://earth.google.com/kml/2.2}Document')
            else:
                print("No Document found in KML")
                continue

            for folder in document.Folder:
                for placemark in folder.Placemark:
                    name = placemark.name.text if hasattr(placemark, 'name') else None
                    description = parse_extended_data(placemark.ExtendedData) if hasattr(placemark, 'ExtendedData') else None
                    if hasattr(placemark, 'Point'):
                        coordinate = parse_coordinates(placemark.Point.coordinates.text)
                    elif hasattr(placemark, 'LineString'):
                        coordinate = parse_coordinates(placemark.LineString.coordinates.text)
                    elif hasattr(placemark, 'Polygon'):
                        coordinate = parse_coordinates(placemark.Polygon.outerBoundaryIs.LinearRing.coordinates.text)
                    else:
                        coordinate = None
                    names.append(name)
                    descriptions.append(description)
                    coordinates.append(coordinate)
                    additional_data.append(description)

       
        df = pd.DataFrame({
            'Name': names,
            'Description': descriptions,
            'Coordinates': coordinates
        })

        print(f"DataFrame created with {len(df)} rows")

     
        if additional_data:
            for data in additional_data:
                if data:
                    for key in data.keys():
                        if key not in df.columns:
                            df[key] = None
                    for i, data_dict in enumerate(additional_data):
                        if data_dict:
                            for key, value in data_dict.items():
                                df.at[i, key] = value

     
        output_file_path = os.path.join(output_directory, os.path.basename(kml_file_path).replace('.kml', '.csv'))
        df.to_csv(output_file_path, index=False)
        print(f"CSV file saved: {output_file_path}")
    except Exception as e:
        print(f"Error processing file {kml_file_path}: {e}")

In [73]:



file_path = '../input/Verra_Projects/komaza/Komaza_Carbon_Project_Farms.csv'


komaza_df = pd.read_csv(file_path)


komaza_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23197 entries, 0 to 23196
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         0 non-null      float64
 1   Description  23197 non-null  object 
 2   Coordinates  0 non-null      float64
 3   kcode        23197 non-null  object 
 4   PlantYear    23197 non-null  int64  
 5   layer        23197 non-null  object 
 6   path         23197 non-null  object 
 7   Region       5386 non-null   object 
 8   Species      5380 non-null   object 
 9   Validity     5386 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 1.8+ MB


In [70]:
komaza_df.head()

,Name,Description,Coordinates,kcode,PlantYear,layer,path,Region,Species,Validity
0,NaN,"{'kcode': 'klf-2019-3519', 'PlantYear': '2019'...",NaN,klf-2019-3519,2019,LR2017_LR2019CoastalKenyaFarms,G:/My Drive/Carbon Project/Carbon Project KML/...,NaN,NaN,NaN
1,NaN,"{'kcode': 'klf-2019-4892', 'PlantYear': '2019'...",NaN,klf-2019-4892,2019,LR2017_LR2019CoastalKenyaFarms,G:/My Drive/Carbon Project/Carbon Project KML/...,NaN,NaN,NaN
2,NaN,"{'kcode': 'klf-2019-2910', 'PlantYear': '2019'...",NaN,klf-2019-2910,2019,LR2017_LR2019CoastalKenyaFarms,G:/My Drive/Carbon Project/Carbon Project KML/...,NaN,NaN,NaN
3,NaN,"{'kcode': 'klf-2019-5427', 'PlantYear': '2019'...",NaN,klf-2019-5427,2019,LR2017_LR2019CoastalKenyaFarms,G:/My Drive/Carbon Project/Carbon Project KML/...,NaN,NaN,NaN
4,NaN,"{'kcode': 'klf-2019-2271', 'PlantYear': '2019'...",NaN,klf-2019-2271,2019,LR2017_LR2019CoastalKenyaFarms,G:/My Drive/Carbon Project/Carbon Project KML/...,NaN,NaN,NaN


Renaming columns

In [74]:
# Renaming the columns to follow our naming format in the paper columns section
columns_rename_mapping = {
    'kcode': 'project_id_reported',
    'kcode': 'site_id_reported',
    #'countPlanted': 'trees_planted_reported',
    'Description': 'description_reported',
    'PlantYear': 'planting_date_reported',
    'Coordinates': 'geometry_reported',
    'path': 'url',
    'Species':'species_planted_reported',
    # 'surfaceAreaKm2': 'site_sqkm',
    'Region': 'country'
    #'survivalRate': 'survival_rate_reported',
}


columns_to_keep = ['kcode', 'Description', 'PlantYear', 'Coordinates', 'path', 'Region','Species']


komaza_df = komaza_df[columns_to_keep]


komaza_df.rename(columns=columns_rename_mapping, inplace=True)

Adding the other defined columns

In [75]:

komaza_df  = komaza_df .assign(site_sqkm=None,species_count_reported=None, survival_rate_reported=None,trees_planted_reported=None)
komaza_df .info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23197 entries, 0 to 23196
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   site_id_reported          23197 non-null  object 
 1   description_reported      23197 non-null  object 
 2   planting_date_reported    23197 non-null  int64  
 3   geometry_reported         0 non-null      float64
 4   url                       23197 non-null  object 
 5   country                   5386 non-null   object 
 6   species_planted_reported  5380 non-null   object 
 7   site_sqkm                 0 non-null      object 
 8   species_count_reported    0 non-null      object 
 9   survival_rate_reported    0 non-null      object 
 10  trees_planted_reported    0 non-null      object 
dtypes: float64(1), int64(1), object(9)
memory usage: 1.9+ MB


In [ ]:

komaza_df .to_csv('../input/Verra/Komaza.csv', index=False)

### Asia Projects

In [8]:
import xml.etree.ElementTree as ET
import csv


tree = ET.parse('/Users/angela/Downloads/Forest_Monitoring/input/Verra_Projects/ASIA/05-Lanzhou AR_Project boundary.kml')
root = tree.getroot()

namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

# Opening and writting file
with open('/Users/angela/Downloads/Forest_Monitoring/input/Verra_Projects/ASIA/05-Lanzhou AR_Project boundary.csv', mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['area', 'coordinates'])  

   
    for placemark in root.findall('.//kml:Placemark', namespace):
      
        area = placemark.find('.//kml:SimpleData[@name="area"]', namespace)
       
        coordinates = placemark.find('.//kml:coordinates', namespace)
        
        if area is not None and coordinates is not None:
            writer.writerow([area.text, coordinates.text.strip()])  # Write the area and coordinates to the CSV file

In [9]:

file_path = '../input/Verra_Projects/ASIA/05-Lanzhou AR_Project boundary.csv'


asia_df = pd.read_csv(file_path)


asia_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41059 entries, 0 to 41058
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   area         41059 non-null  float64
 1   coordinates  41059 non-null  object 
dtypes: float64(1), object(1)
memory usage: 641.7+ KB


In [10]:
asia_df.head()

,area,coordinates
0,410.187367,"104.390174248,35.9628743625646,0 104.390158098..."
1,883.465773,"104.392687327499,35.9627637278053,0 104.392692..."
2,736.898031,"104.373402696529,35.956069517541,0 104.3733422..."
3,975.749717,"104.37662814091,35.9474573229517,0 104.3764530..."
4,364.117563,"104.378591297954,35.9528386376154,0 104.378591..."


In [11]:

asia_df['area'] = pd.to_numeric(asia_df['area'], errors='coerce')


asia_df['area'].fillna(0, inplace=True)


asia_df['area (km²)'] = asia_df['area'] / 100

/var/folders/_l/gj90rjl90fngb4_8ph5x5jhc0000gn/T/ipykernel_1935/2883943885.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  asia_df['area'].fillna(0, inplace=True)


In [13]:
# Renaming the columns to follow our naming format in the paper columns section
columns_rename_mapping = {

    'coordinates': 'geometry_reported',
    'area (km²)': 'site_sqkm',
    #'survivalRate': 'survival_rate_reported',
}

asia_df = asia_df[list(columns_rename_mapping.keys())]


asia_df.rename(columns=columns_rename_mapping, inplace=True)

/var/folders/_l/gj90rjl90fngb4_8ph5x5jhc0000gn/T/ipykernel_1935/3105024638.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asia_df.rename(columns=columns_rename_mapping, inplace=True)


In [16]:
asia_df['site_id_reported'] = ['tist_asia_{}'.format(i) for i in range(1, len(asia_df) + 1)]
asia_df = asia_df.assign(species_count_reported=None, species_planted_reported=None, survival_rate_reported=None,trees_planted_reported=None,planting_date_reported=None,description_reported=None,country=None,project_id_reported=None)
asia_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41059 entries, 0 to 41058
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   geometry_reported         41059 non-null  object 
 1   site_sqkm                 41059 non-null  float64
 2   site_id_reported          41059 non-null  object 
 3   species_count_reported    0 non-null      object 
 4   species_planted_reported  0 non-null      object 
 5   survival_rate_reported    0 non-null      object 
 6   trees_planted_reported    0 non-null      object 
 7   planting_date_reported    0 non-null      object 
 8   description_reported      0 non-null      object 
 9   country                   0 non-null      object 
 10  project_id_reported       0 non-null      object 
dtypes: float64(1), object(10)
memory usage: 3.4+ MB


In [ ]:
asia_df.to_csv('../input/Verra/Asia.csv', index=False)